In [1]:
import os
os.chdir('..')

In [ ]:
import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor, as_completed
from datetime import timedelta

import polars as pl

import config
from aws.kline.util import local_list_kline_symbols
from config import BINANCE_DATA_DIR, TradeType
from merge.kline import merge_klines, merge_and_split_gaps, scan_gaps, fill_kline_gaps

pl.Config.set_tbl_rows(30)

In [ ]:

symbol = 'BNXUSDT'
time_interval = '1m'

min_delta = 1
min_price_chg = 0.1

df = merge_klines(TradeType.spot, "BNXUSDT", time_interval, exclude_empty=True)
display(scan_gaps(df, min_delta, min_price_chg))

df = merge_klines(TradeType.spot, "BTCUSDT", time_interval, exclude_empty=True)
display(scan_gaps(df, min_delta, min_price_chg))

In [ ]:
symbol = 'BNXUSDT'
time_interval = '1m'

min_delta = 1
min_price_chg = 0.1

df = merge_klines(TradeType.spot, "BTCUSDT", time_interval, exclude_empty=True)
df1 = fill_kline_gaps(df, time_interval)
df1.filter(pl.col('volume') == 0).tail(30)

In [ ]:
symbol = 'BNXUSDT'
time_interval = '1m'

min_days = 1
min_price_chg = 0.1

results = merge_and_split_gaps(
    trade_type=TradeType.spot,
    time_interval=time_interval,
    symbol=symbol,
    min_days=min_days,
    min_price_chg=min_price_chg,
    exclude_empty=True,
)


for k, df in results.items():
    print(k)
    display(df.head())
    display(df.tail())


In [ ]:
symbol = 'NBTUSDT'
time_interval = '1m'

min_days = 1
min_price_chg = 0.1

results = merge_and_split_gaps(
    trade_type=TradeType.spot,
    time_interval=time_interval,
    symbol=symbol,
    min_days=min_days,
    min_price_chg=min_price_chg,
    exclude_empty=True,
)
for k, df in results.items():
    print(k)
    display(df.head())
    display(df.tail())


In [7]:
def display_gaps_all(trade_type: TradeType, time_interval: str, min_delta: timedelta, min_price_chg: float):
    symbols = local_list_kline_symbols(trade_type, time_interval)

    dfs = []
    for symbol in symbols:
        ldf = merge_klines(TradeType.spot, symbol, time_interval, exclude_empty=True)
        df_gap = scan_gaps(ldf, min_delta, min_price_chg)
        if not df_gap.is_empty():
            dfs.append(df_gap.with_columns(pl.lit(symbol).alias("symbol")))

    df_gaps =pl.concat(dfs).sort('candle_begin_time')
    display(df_gaps)

# display_gaps_all(TradeType.spot, '1m', timedelta(days=1), 0.1)
# display_gaps_all(TradeType.spot, '1m', timedelta(days=2), 0)
# display_gaps_all(TradeType.spot, '1m', timedelta(days=1), 0)

# display_gaps_all(TradeType.um_futures, '1m', timedelta(days=1), 0.1)
# display_gaps_all(TradeType.um_futures, '1m', timedelta(days=2), 0)
# display_gaps_all(TradeType.um_futures, '1m', timedelta(days=1), 0)